In [6]:
"""
-Arman Bazarchi-
Temporal_Landing
here we get the raw data(image and metadata about each image) 
from source (imageomics/TreeOfLife-200M) and store in our temporal-zone.
miniIO user must be "admin", and password "password123".
as data is huge 200m, we get the data by chunks of 1% out of 100% of all data.
so we limit the amount of getting data on each run.
user is asked for a maximum number of species per family, limiting data to store up to a defined  
number of species for each stored family.
user also is asked for a maximum number of observations(images) per each stored species.
also is asked for the chunks interval for the run, for example 5-10 will get from 5th% up to 10th% of the whole 100% of data.
for now only retrieves images of snake families only because of not being able to store a lot of data at the moment.

then code connects to minIO create a temporal-zone bucket and a subbucket 'temporal-landing', 
gets defined interval % of the train split of data,
stores images in a folder 'images' and a csv file of the needed metadata in a folder 'metadata',
it checks and avoids storing duplicates in temporal-zone, also deletes any temporary file and cache
and huggingface heavy cach from local storage.


# Login using e.g. `huggingface-cli login` to access this dataset
"""

from datasets import load_dataset
from datasets import config
import shutil
from minio import Minio
from PIL import Image
from tqdm import tqdm
import io
import pandas as pd
import os
import time
import requests
import re

# ==============================
# 1 Configuration
# ==============================
def process_landing_zone(
    MINIO_ENDPOINT = "localhost:9000",
    ACCESS_KEY = "admin",
    SECRET_KEY = "password123"):

    ROOT_BUCKET = "temporal-zone"         
    TEMP_PREFIX = "temporal-landing"     # Subbucket for temporal
    
    # Metadata config
    METADATA_REMOTE_PATH = f"{TEMP_PREFIX}/metadata/metadata_final.csv"
    METADATA_LOCAL = "metadata_final.csv"
    
    # Get user input for parameters
    print("Please enter the following parameters:")
    MAX_PER_SPECIES = int(input("MAX_PER_SPECIES (e.g., 30): "))
    MAX_SPECIES_PER_FAMILY = int(input("MAX_SPECIES_PER_FAMILY (e.g., 11): "))
    chunk_interval = input("Chunk interval (e.g., '5-10'): ")
    
    # Parse chunk interval
    start_chunk, end_chunk = map(int, chunk_interval.split('-'))
    
    # Global Limit
    MAX_TOTAL_IMAGES = 100000
    
    # Known snake families
    SNAKE_FAMILIES = {
        "Viperidae", "Elapidae", "Colubridae", "Pythonidae", "Boidae",
        "Typhlopidae", "Lamprophiidae", "Natricidae", "Dipsadidae",
        "Leptotyphlopidae", "Xenopeltidae", "Anomalepididae", "Loxocemidae",
        "Uropeltidae", "Cylindrophiidae", "Aniliidae", "Acrochhordidae",
        "Anomochilidae", "Atractaspididae", "Bolyeridae"
    }
    
    
    #  Initialize MinIO client
    client = Minio(
        MINIO_ENDPOINT,
        access_key=ACCESS_KEY,
        secret_key=SECRET_KEY,
        secure=False
    )
    
    
    #  Ensure temporal-landing and temporal-zone exist
    if not client.bucket_exists(ROOT_BUCKET):
        client.make_bucket(ROOT_BUCKET)
        print(f" Created top-level bucket: {ROOT_BUCKET}")
    
    # Create temporal-landing subfolder if missing
    temporal_exists = any(
        obj.object_name.startswith(f"{TEMP_PREFIX}/")
        for obj in client.list_objects(ROOT_BUCKET, recursive=False)
    )
    if not temporal_exists:
        client.put_object(
            ROOT_BUCKET,
            f"{TEMP_PREFIX}/.init",
            data=io.BytesIO(b"init"),
            length=4,
            content_type="text/plain"
        )
        print(f"✅ Created subfolder: {TEMP_PREFIX}/ inside {ROOT_BUCKET}")
    
    # ==============================
    # 2 Check for existing metadata
    # ==============================
    try:
        client.fget_object(ROOT_BUCKET, METADATA_REMOTE_PATH, METADATA_LOCAL)
        metadata_df = pd.read_csv(METADATA_LOCAL)
        print(f" Loaded metadata from {TEMP_PREFIX} with {len(metadata_df)} records.")
    
    except Exception:
        metadata_df = pd.DataFrame(columns=[
            "uuid","temporal_path", "kingdom", "phylum", "class", "order", "family",
            "genus", "species", "scientific_name", "common", "image_url"
        ])
        print(" No metadata found in temporal-landing — starting fresh.")
    # Maintain fast lookup sets
    existing_uuids = set(metadata_df["uuid"].dropna().unique()) if "uuid" in metadata_df.columns else set()
    
    # ==============================
    # 3 Scan existing images
    # ==============================
    print(" Scanning existing images in temporal-landing...")
    
    existing_image_ids = set()
    for obj in client.list_objects(ROOT_BUCKET, prefix=f"{TEMP_PREFIX}/images/", recursive=True):
        # Extract UUID from filename: temporal-landing/images/<uuid>.jpg
        match = re.match(rf"{TEMP_PREFIX}/images/([a-f0-9\-]+)\.jpg", obj.object_name, re.IGNORECASE)
        if match:
            existing_image_ids.add(match.group(1))
    
    print(f" Found {len(existing_image_ids)} existing image files in temporal-landing.")
    
    family_species = {}
    species_counts = {}
    
    # ==============================
    # 4 Build family -> species map from metadata
    # ==============================
    if not metadata_df.empty:
        for _, row in metadata_df.iterrows():
            fam = row.get("family")
            sp = row.get("species")
            if pd.notna(fam) and pd.notna(sp):
                family_species.setdefault(fam, set()).add(sp)
                species_counts[sp] = species_counts.get(sp, 0) + 1
    
    print(f" Tracking {len(family_species)} families from existing metadata.")
    
    # ==============================
    # 5 Progressive Chunk Loop
    # ==============================
    for chunk_idx in range(start_chunk, end_chunk):
        print(f"\n Processing chunk {chunk_idx}%–{chunk_idx+1}% ...")
    
        try:
            dataset = load_dataset(
                "imageomics/TreeOfLife-200M",
                split=f"train[{chunk_idx}%:{chunk_idx+1}%]")  # <-- avoids writing huge Arrow files
        except Exception as e:
            print(f" Skipping chunk {chunk_idx}: {e}")
            continue
    
        # getting needed metadata
        for sample in tqdm(dataset):
            species = sample.get("species") or "unknown"
            kingdom = sample.get("kingdom")
            cls = sample.get("class")
            order = sample.get("order")
            family = sample.get("family") or "unknown"
            img_id = sample.get("uuid")
            image_url = sample.get("source_url")
    
            # Only save snake images for now
            if not (
                kingdom and kingdom.strip().lower() == "animalia"
                and cls and cls.strip().lower() == "squamata"
                and family in SNAKE_FAMILIES
            ):
                continue
    
            # skip if already have enough per this specie
            count = species_counts.get(species, 0)
            if count >= MAX_PER_SPECIES:
                continue
    
            # skip if already have enough specie for this family
            current_species_in_family = family_species.get(family, set())
            if len(current_species_in_family) >= MAX_SPECIES_PER_FAMILY and species not in current_species_in_family:
                continue
    
            try:
                if img_id not in existing_image_ids:
                    response = requests.get(image_url, timeout=10)
                    response.raise_for_status()
                    image_bytes = response.content
    
                    img = Image.open(io.BytesIO(image_bytes))
                    img.verify()
                    buffer = io.BytesIO(image_bytes)
    
                    # Save to MinIO 
                    object_name = f"{TEMP_PREFIX}/images/{img_id}.jpg"
                    client.put_object(
                        ROOT_BUCKET,
                        object_name,
                        data=buffer,
                        length=len(image_bytes),
                        content_type="image/jpeg"
                    )
                    existing_image_ids.add(img_id)
    
                # skip duplicates
                if img_id not in existing_uuids:
                    # Save needed metadata
                    record = {
                        "uuid": img_id,
                        "temporal_path": object_name,
                        "kingdom": kingdom,
                        "phylum": sample.get("phylum"),
                        "class": cls,
                        "order": order,
                        "family": family,
                        "genus": sample.get("genus"),
                        "species": species,
                        "scientific_name": sample.get("scientific_name"),
                        "common": sample.get("common"),
                        "image_url": image_url
                    }
    
                    metadata_df = pd.concat([metadata_df, pd.DataFrame([record])], ignore_index=True)
                    metadata_df.to_csv(METADATA_LOCAL, index=False)
                    client.fput_object(ROOT_BUCKET, METADATA_REMOTE_PATH, METADATA_LOCAL, content_type="text/csv")
    
                    species_counts[species] = count + 1
                    family_species.setdefault(family, set()).add(species)
                    existing_uuids.add(img_id)
    
                    print(f" Saved {species} ({species_counts[species]}/{MAX_PER_SPECIES}) "
                          f"in family {family} ({len(family_species.get(family, []))}/{MAX_SPECIES_PER_FAMILY})")
    
            except Exception as e:
                print(f" Error saving {species}: {e}")
                continue
    
            if len(metadata_df) >= MAX_TOTAL_IMAGES:
                print(" Reached global image limit.")
                break
    
        if len(metadata_df) >= MAX_TOTAL_IMAGES:
            break
    
        time.sleep(1)
    
    print(f" Finished progressive sampling. Total images: {len(metadata_df)}")
    
    # ==============================
    # 6 Cleanup local CSV
    # ==============================
    if os.path.exists(METADATA_LOCAL):
        os.remove(METADATA_LOCAL)
        print(f"🗑️ Removed local metadata file: {METADATA_LOCAL}")
    
process_landing_zone();

Please enter the following parameters:


MAX_PER_SPECIES (e.g., 30):  1
MAX_SPECIES_PER_FAMILY (e.g., 11):  1
Chunk interval (e.g., '5-10'):  1-2


 Loaded metadata from temporal-landing with 15 records.
 Scanning existing images in temporal-landing...
 Found 0 existing image files in temporal-landing.
 Tracking 14 families from existing metadata.

 Processing chunk 1%–2% ...


Generating train split:   0%|          | 0/213937319 [00:00<?, ? examples/s]

  0%|                                                                          | 2649/2139373 [00:02<37:15, 955.89it/s]

 Saved imperator (1/1) in family Boidae (1/1)


100%|██████████████████████████████████████████████████████████████████████| 2139373/2139373 [06:57<00:00, 5122.29it/s]


 Finished progressive sampling. Total images: 16
🗑️ Removed local metadata file: metadata_final.csv


In [7]:
import os
import shutil
import tempfile
import subprocess
import time
from datasets import config

"""
  Here we notice that getting data from huggingface stores heavy size of cache of that dataset on local storage.
  so we try to safely remove this cache and free up space.
"""

hf_cache = config.HF_DATASETS_CACHE

def delete_cache(path):
    # Rename and delete the Hugging Face cache in a background process.
    # as windows denies access and stops deleting the cache, it must run in a background process.
    if not os.path.exists(path):
        print(" No Hugging Face cache found — skipping.")
        return

    temp_parent = tempfile.gettempdir()
    temp_target = os.path.join(temp_parent, f"hf_cache_to_delete_{int(time.time())}")

    try:
        os.rename(path, temp_target)
        print(f"  Renamed cache folder to {temp_target}")
    except Exception as e:
        print(f" Could not rename cache folder: {e}")
        return

    # Launch background deletion process, after 3 seconds of wait
    delete_cmd = f'timeout /T 3 >NUL & rmdir /S /Q "{temp_target}"'
    try:
        subprocess.Popen(["cmd", "/c", delete_cmd], creationflags=subprocess.DETACHED_PROCESS)
        print(" Scheduled background cache deletion.")
    except Exception as e:
        print(f" Could not schedule background deletion: {e}")

# Run cleanup
print(f"Deleting Hugging Face datasets cache at {hf_cache} ...")
delete_cache(hf_cache)

# However some binary files from online huggingface hub stay in locat storage



Deleting Hugging Face datasets cache at C:\Users\Arman\.cache\huggingface\datasets ...
  Renamed cache folder to C:\Users\Arman\AppData\Local\Temp\hf_cache_to_delete_1761522365
 Scheduled background cache deletion.
